In [4]:
#!pip install scikit-learn

In [5]:
#!pip install fastapi "uvicorn[standard]" scikit-learn pandas

In [6]:
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [7]:
# Exemplo de dados
data = {
    "idade": [25, 45, 30, 50, 40, 35, 28, 52, 33, 41],
    "genero": ["M","F","M","F","M","M","F","F","M","F"],
    "tempo_de_contrato_meses": [12, 24, 5, 36, 48, 20, 6, 50, 15, 22],
    "gasto_mensal": [120, 200, 80, 300, 150, 110, 95, 310, 100, 180],
    "churn": [0,1,0,1,0,0,1,1,0,1]
}
df = pd.DataFrame(data)

X = df.drop(columns=["churn"])
y = df["churn"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Definição de features
num_features = ["idade", "tempo_de_contrato_meses", "gasto_mensal"]
cat_features = ["genero"]

# Pré-processamento
preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_features),
        ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), cat_features),
    ]
)

# Pipeline completo
clf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", LogisticRegression(max_iter=1000))
])

# Treino
clf.fit(X_train, y_train)

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [10]:
train_score = clf.score(X_train, y_train)
test_score = clf.score(X_test, y_test)
print("train", train_score)
print("test", test_score)
#print(classification_report(y_test, clf.predict(X_test)))
data[' probability'] = clf.predict_proba(X)[:, 1]

train 1.0
test 1.0


In [11]:
# Create LIME Explainer
feature_names = list(ce.columns_)
categorical_features = list(ce.cat_columns_ix_.values())
categorical_names = {i: ce.classes_[c]
                     for c, i in ce.cat_columns_ix_.items()}
class_names = ['No ' + labels.name, labels.name]
explainer = LimeTabularExplainer(ce.transform(data),
                                 feature_names=feature_names,
                                 class_names=class_names,
                                 categorical_features=categorical_features,
                                 categorical_names=categorical_names)

NameError: name 'ce' is not defined

In [12]:
explainedmodel = ExplainedModel(data=data, labels=labels, model_name='telco_linear',
                                categoricalencoder=ce, pipeline=pipe,
                                explainer=explainer, data_dir=data_dir)
explainedmodel.save()

NameError: name 'ExplainedModel' is not defined

In [ ]:

# Salvar pipeline
with open("pipeline.pkl", "wb") as f:
    pickle.dump(clf, f)